In [3]:
import numpy as np
import pandas as pd
import os

from dataloader import get_data, get_weather, get_hod, get_dow

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [4]:
data_set = 'Irish_2010'
year = '2010'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

100%|██████████| 918/918 [00:28<00:00, 32.52it/s]


In [5]:
month = 1
n_clusters = 2
method = 'hierarchical/euclidean'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)

series = data[:, month-1, :months[month-1]*24]
weather = get_weather(path, data_set, month)
week = get_dow(data_set, month)
day = get_hod(month)

In [ ]:
error_train = []
error_test = []

for i in range(1, n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    sub_series = np.sum(sub_series, axis=0)
    
    total_series = np.vstack((sub_series, weather, week, day))
    
    test = total_series[:, -168:]
    train = total_series[:, :-168]
    
    scale = np.zeros(2)
    scale[0] = np.max(train[0])
    scale[1] = np.min(train[0])
    train[0] = (train[0] - scale[1]) / (scale[0] - scale[1])
    test[0] = (test[0] - scale[1]) / (scale[0] - scale[1])
    
    lag = 24
    d = 1

    trainX, trainY = get_train_set_qrnn(train, lag, d)
    testX, testY = get_test_set_qrnn(train, test, lag, d)

    break

## QRA (Step 1)

In [6]:
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.


In [22]:
# Build linear model
input_dim = 100
model = Sequential([Dense(1, input_shape=(input_dim,))])

In [23]:
# Train
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, Ytrain, epochs=10)

## QRA (Step 2)

In [8]:
def quantile_loss(y_true, y_pred, q):
    tmp1 = (q / 100 - 1) * (y_true - y_pred)
    tmp2 = q / 100 * (y_true - y_pred)
    return K.mean(K.maximum(tmp1, tmp2))

In [11]:
# Build qr model
input_dim = 8
model = Sequential([Dense(1, input_shape=(input_dim,))])

In [ ]:
# Train
model.compile(loss=lambda y_true, y_pred: quantile_loss(y_ture, y_pred, q), optimizer='adam')